# Pricing Model for Client

### Set up the parameters

In [1]:
import numpy as np
import pandas as pd

# For data visualization
import matplotlib.pyplot as plt

# For date handling
from datetime import datetime

### This is the sample date and params

In [2]:
inject_rate = 1e-1
withdraw_rate = 1e-1 # dollar per MMBtu
injection_date = {'10-31-2020': 100000, '01-31-2022': 20000} # Dictionary with key is date and value is MMBtu injecting
withdraw_date = {'06-30-2021': 50000, '11-30-2021': 50000} # Dictionary with key is date and value is MMBtu withdrawing
maximum_storage = 1e6 # MMBtu
storage_cost = 3e-2 # dollar per MMbtu per month

### Step 1: Create a dictionary to lookup price based on the forecast output last task

In [3]:
df = pd.read_csv("forecast_output.csv") # forecast ouput is the predicted gas price we made in Task 1


def create_price_lookup(df):
    price_dict = {}

    for date, price in zip(df['Dates'], df['Prices']):
        # Convert the date string to datetime object using the same format as date_sorting
        datetime_key = datetime.strptime(date, "%m-%d-%Y")
        price_dict[datetime_key] = price
    
    # Sort the dictionary by date to maintain consistency with date_sorting
    return price_dict

price_lookup = create_price_lookup(df)

### Step 2: Verify valid data & Create a transaction history

We verify the data to see if it is valid. Then for each injection/withdrawal happened, save it in a `transaction_history`
The meaning of storing it is for easier cost calculation in the future. After validate the information, we will also create a `storage_history` to remember the total storage after each transaction. This is for calculating storage cost.

In [5]:
def date_sorting(injection_date, withdraw_date):
    combined_dict = {}

    for date, value in injection_date.items():
        combined_dict[datetime.strptime(date, "%m-%d-%Y")] = value

    # Add withdrawal dates (negative values)
    for date, value in withdraw_date.items():
        combined_dict[datetime.strptime(date, "%m-%d-%Y")] = -value
    
    # Sort the dictionary by date
    sorted_dict = dict(sorted(combined_dict.items(), key=lambda item: item[0]))

    return sorted_dict

transaction_history = date_sorting(injection_date, withdraw_date)

def validate_information(injection_date, withdraw_date, maximum_storage):
    current_storage = 0
    storage_history = {}
    valid = True

    for date, value in date_sorting(injection_date, withdraw_date).items():
        current_storage += value

        if current_storage > maximum_storage:
            raise ValueError("Storage capacity exceeded at date: " + date)

        if current_storage < 0:
            raise ValueError("Storage capacity is negative at date: " + date)
        
        storage_history.update({date: current_storage})
    
    return storage_history, valid

storage_history, valid = validate_information(injection_date, withdraw_date, maximum_storage)
if valid:
    print("Information is valid")
    print("Storage history: ", storage_history)

Information is valid
Storage history:  {datetime.datetime(2020, 10, 31, 0, 0): 100000, datetime.datetime(2021, 6, 30, 0, 0): 50000, datetime.datetime(2021, 11, 30, 0, 0): 0, datetime.datetime(2022, 1, 31, 0, 0): 20000}


### Step 3: Calculate storage cost at each time

After having the `storage_history`, we calculate the storage cost at each time in the history to further calculate the total cost.

In [6]:
def storage_cost_history(storage_history, storage_cost):
    storage_cost_history = {}

    for date, storage in storage_history.items():
        storage_cost_history[date] = storage * storage_cost

    return storage_cost_history

storage_cost_history = storage_cost_history(storage_history, storage_cost)
print("Storage cost history: ", storage_cost_history)

Storage cost history:  {datetime.datetime(2020, 10, 31, 0, 0): 3000.0, datetime.datetime(2021, 6, 30, 0, 0): 1500.0, datetime.datetime(2021, 11, 30, 0, 0): 0.0, datetime.datetime(2022, 1, 31, 0, 0): 600.0}


Here we calculate the total cost and store it in `total_storage_cost`

In [8]:
def calculate_total_storage_cost(storage_cost_history):
    total_cost = 0

    date = list(storage_cost_history.keys())

    for i in range(len(date) - 1):
        current_date = date[i]
        next_date = date[i + 1]
        monthly_cost = storage_cost_history[current_date]

        storage_duration = (next_date.year - current_date.year) * 12 + (next_date.month - current_date.month)

        total_cost += monthly_cost * storage_duration
    
    return total_cost

total_storage_cost = calculate_total_storage_cost(storage_cost_history)
print("Total storage cost: ", total_storage_cost)

Total storage cost:  31500.0


### Step 4: Calculate injection and withdrawal cost

Based on the `transaction_history` and `inject_rate`.`withdraw_rate`. We calculate the total cost we will use for injection and withdrawal. Computing this value separately brought us spaces for improvements when the inject and withdraw rate is not constant.

In [10]:
def calculate_inject_withdraw_cost(injection_date, withdraw_date, inject_rate, withdraw_rate):
    total_cost = 0

    for date, value in date_sorting(injection_date, withdraw_date).items():
        if value > 0:
            total_cost += value * inject_rate
        else:
            total_cost += -value * withdraw_rate

    return total_cost

inject_withdraw_cost = calculate_inject_withdraw_cost(injection_date, withdraw_date, inject_rate, withdraw_rate)
print("Injection and withdrawal cost: ", inject_withdraw_cost)

Injection and withdrawal cost:  22000.0


### Step 5: Calculate gross profit

Calculate the gross profit we got from trading, not taking in consideration any cost such as storage, inject or withdraw cost.

In [11]:
def calculate_gross_profit(price_lookup, transaction_history):
    gross_profit = 0

    for date, value in transaction_history.items():
        price = price_lookup[date]

        if value > 0:
            gross_profit -= value * price
        else:
            gross_profit += value * price

    return gross_profit

gross_profit = calculate_gross_profit(price_lookup, transaction_history)
print("Gross profit: ", gross_profit)

Gross profit:  -2269500.0


### Final: Compute the net profit of our strategy

In [12]:
net_profit = gross_profit - total_storage_cost - inject_withdraw_cost
print("Net profit: ", net_profit)

Net profit:  -2323000.0
